# 🍀 はじめに 🍀
# 😊いつもチャンネルをご覧いただいている皆様、ご視聴ありがとうございます！  
### 🌱このノートブックは主に、生成AIツール活用の研究目的のために制作しました。
### 🌱使用目的は「ComfyUI」上で「Wan2.2」等のツールを高価なPC不要でも使用可能にする為に制作したものです。
### 🌱各種生成ツールのコツなど詳しい使い方については、私のYoutbeチャンネルの中で解説しています。
### 🌱より便利で汎用的なノートブックやＷＦも「メンバーシップ特典」として配布しています。
### 📺『ＡＩみちくさｃｈ』 (https://www.youtube.com/channel/UC84fyKjiilxssZVxhE_RiaA)
####　🍀制作者：ざすこ（道草 雑草子） Xアカウント：https://x.com/zasuko_michiksa
##### 🥰このノートブックが役に立ったよ～！と思った方は、チャンネル登録や、メンバーシップのご参加をご検討頂けますと幸いです。
#＜このnotebookの使い方の解説動画＞
### ※現在、鋭意制作中（2025/10/22現在） もう少しお待ちください💧

# 🥚各種モデルのDL ＆ ComfyUIの起動🐣
##＜はじめる前の準備＞※重要※
### ❶生成に必要な各種モデルファイルのDL用URLを格納する各フォルダの記入欄に記載して下さい。
└🌐GoogleColab上の 📂ComfyUI / models  以下に、生成に必要な各種ファイルをDLします。（DL容量が少ない程起動が速くなります。）
### ❷（▶）ボタンをクリックしてComfyUIを起動


In [ ]:
# @markdown # 👈この（▶）ボタンをクリックしてComfyUIを起動（☕起動まで約5～6分）
# @markdown ### ※停止と再起動もこのボタンで行います。
# @markdown ## 🌱スクロール下の実行ログに青字で👇「https～ localhost:8188」👀が表示されたら🖱️クリックしてください。

!pip install torch==2.8.0 torchvision==0.23.0

# using_L4_GPU = True # @param {type:"boolean"}
using_L4_GPU = False
include_manager = True # @param {type:"boolean"}

%cd /content
from IPython.display import clear_output
clear_output()
!pip install -q torchsde einops diffusers accelerate xformers==0.0.32.post1 triton==3.4 sageattention
!pip install av spandrel albumentations onnx opencv-python onnxruntime
!pip install color-matcher
!pip install onnxruntime-gpu -y
!git clone https://github.com/comfyanonymous/ComfyUI
!pip install -r /content/ComfyUI/requirements.txt
clear_output()

%cd /content/ComfyUI/custom_nodes
# !git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git
if include_manager:
    !git clone https://github.com/ltdrdata/ComfyUI-Manager
!git clone --branch forQwen https://github.com/Isi-dev/ComfyUI_GGUF.git
!git clone https://github.com/Isi-dev/ComfyUI_DeleteModelPassthrough.git
!git clone https://github.com/Isi-dev/comfyui_controlnet_aux
!git clone https://github.com/kijai/ComfyUI-WanVideoWrapper
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/kijai/ComfyUI-segment-anything-2
!git clone https://github.com/kijai/ComfyUI-Florence2
!git clone https://github.com/john-mnz/ComfyUI-Inspyrenet-Rembg.git
!git clone https://github.com/Isi-dev/ComfyUI_Animation_Nodes_and_Workflows
%cd /content/ComfyUI/custom_nodes/ComfyUI_GGUF
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI_DeleteModelPassthrough
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/comfyui_controlnet_aux
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-WanVideoWrapper
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-VideoHelperSuite
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-KJNodes
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-Florence2
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-Inspyrenet-Rembg
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI_Animation_Nodes_and_Workflows
!pip install -r requirements.txt
if include_manager:
    %cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
    !pip install -r requirements.txt

clear_output()

%cd /content/ComfyUI

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import subprocess
import sys
from pathlib import Path

def install_apt_packages():
    packages = ['aria2']

    try:
        # Run apt install silently (using -qq)
        subprocess.run(
            ['apt-get', '-y', 'install', '-qq'] + packages,
            check=True,
            capture_output=True
        )
        print("✓ apt packages installed")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error installing apt packages: {e.stderr.decode().strip() or 'Unknown error'}")

print("Installing apt packages...")
install_apt_packages()

def download_with_aria2c(link, folder="/content/ComfyUI/models/loras"):
    import os

    filename = link.split("/")[-1].split('?')[0]  # Remove URL parameters
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d {folder} -o {filename}"

    print("Executing download command:")
    print(command)

    os.makedirs(folder, exist_ok=True)
    get_ipython().system(command)

    return filename

def download_civitai_model(civitai_link, civitai_token, folder="/content/ComfyUI/models/loras"):
    import os
    import time

    os.makedirs(folder, exist_ok=True)

    try:
        model_id = civitai_link.split("/models/")[1].split("?")[0]
    except IndexError:
        raise ValueError("Invalid Civitai URL format. Please use a link like: https://civitai.com/api/download/models/1523247?...")

    civitai_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
    if civitai_token:
        civitai_url += f"&token={civitai_token}"

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"model_{timestamp}.safetensors"

    full_path = os.path.join(folder, filename)

    download_command = f"wget --max-redirect=10 --show-progress \"{civitai_url}\" -O \"{full_path}\""
    print("Downloading from Civitai...")

    os.system(download_command)

    local_path = os.path.join(folder, filename)
    if os.path.exists(local_path) and os.path.getsize(local_path) > 0:
        print(f"LoRA downloaded successfully: {local_path}")
    else:
        print(f"❌ LoRA download failed or file is empty: {local_path}")

    return filename

def download_lora(link, folder="/content/ComfyUI/models/loras", civitai_token=None):
    """
    Download a model file, automatically detecting if it's a Civitai link or huggingface download.

    Args:
        link: The download URL (either huggingface or Civitai)
        folder: Destination folder for the download
        civitai_token: Optional token for Civitai downloads (required if link is from Civitai)

    Returns:
        The filename of the downloaded model
    """
    if "civitai.com" in link.lower():
        if not civitai_token:
            raise ValueError("Civitai token is required for Civitai downloads")
        return download_civitai_model(link, civitai_token, folder)
    else:
        return download_with_aria2c(link, folder)

def model_download(url: str, dest_dir: str, filename: str = None, silent: bool = True) -> bool:
    """
    Colab-optimized download with aria2c

    Args:
        url: Download URL
        dest_dir: Target directory (will be created if needed)
        filename: Optional output filename (defaults to URL filename)
        silent: If True, suppresses all output (except errors)

    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Create destination directory
        Path(dest_dir).mkdir(parents=True, exist_ok=True)

        # Set filename if not specified
        if filename is None:
            filename = url.split('/')[-1].split('?')[0]  # Remove URL parameters

        # Build command
        cmd = [
            'aria2c',
            '--console-log-level=error',
            '-c', '-x', '16', '-s', '16', '-k', '1M',
            '-d', dest_dir,
            '-o', filename,
            url
        ]

        # Add silent flags if requested
        if silent:
            cmd.extend(['--summary-interval=0', '--quiet'])
            print(f"Downloading {filename}...", end=' ', flush=True)

        # Run download
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)

        if silent:
            print("Done!")
        else:
            print(f"Downloaded {filename} to {dest_dir}")
        return filename

    except subprocess.CalledProcessError as e:
        error = e.stderr.strip() or "Unknown error"
        print(f"\nError downloading {filename}: {error}")
        return False
    except Exception as e:
        print(f"\nError: {str(e)}")
        return False

# ========================================
# ダウンロード処理関数
# ========================================

def parse_urls(url_string):
    """
    複数行のURL文字列をパースして、有効なURLのリストを返す

    Args:
        url_string: 改行区切りのURL文字列

    Returns:
        list: 有効なURLのリスト
    """
    urls = []
    for line in url_string.strip().split('\n'):
        line = line.strip()
        # コメント行（#始まり）と空行をスキップ
        if line and not line.startswith('#') and line.startswith('http'):
            urls.append(line)
    return urls

def download_models_to_folder(url_string, folder_path, civitai_token=None):
    """
    指定されたフォルダに複数のモデルをダウンロード

    Args:
        url_string: 改行区切りのURL文字列
        folder_path: ダウンロード先フォルダパス
        civitai_token: Civitai APIトークン（オプション）
    """
    urls = parse_urls(url_string)

    if not urls:
        return

    print(f"\n📂 Downloading to: {folder_path}")
    print(f"   Files to download: {len(urls)}")

    for i, url in enumerate(urls, 1):
        filename = url.split('/')[-1].split('?')[0]
        print(f"   [{i}/{len(urls)}] {filename}...")

        # Civitai URLの場合はdownload_lora関数を使用
        if "civitai.com" in url.lower():
            download_lora(url, folder=folder_path, civitai_token=civitai_token)
        else:
            model_download(url, folder_path)

    print(f"✓ Completed: {folder_path}\n")


# ========================================
# 📥 各種モデルのダウンロード設定
# ========================================

# @markdown # 📥 モデルダウンロード設定
# @markdown 各フォルダごとに、使用したいモデルのダウンロードURLを入力してください。
# @markdown 複数のURLは改行で区切って入力できます。

# @markdown ---
# @markdown ## 📁 diffusion_models フォルダ
# @markdown 各種メインモデル（動画生成など）
diffusion_models_url_1 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors" # @param {type:"string"}
diffusion_models_url_2 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors" # @param {type:"string"}

# 入力されたURLを結合
diffusion_models_urls = "\n".join([
    url for url in [
        diffusion_models_url_1,
        diffusion_models_url_2,

    ] if url.strip()  # 空白のみの入力も除外
])

if diffusion_models_urls:
    download_models_to_folder(diffusion_models_urls, "/content/ComfyUI/models/diffusion_models")

# @markdown ---
# @markdown ## 📁 text_encoders フォルダ
# @markdown テキストエンコーダー（UMT5等）
text_encoders_urls = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 vae フォルダ
# @markdown VAE（変分オートエンコーダー）
vae_urls = "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 clip_vision フォルダ
# @markdown CLIP Visionモデル（画像理解用）
clip_vision_urls = "https://huggingface.co/Isi99999/Wan_Extras/resolve/main/clip_vision_h.safetensors" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 loras フォルダ
# @markdown LoRAモデル（高速化、ライティング調整等）

loras_url_1 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors" # @param {type:"string"}
loras_url_2 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors" # @param {type:"string"}
loras_url_3 = "" # @param {type:"string"}

# URLをまとめる
loras_urls = "\n".join([url for url in [loras_url_1, loras_url_2, loras_url_3] if url])

# @markdown ---
# @markdown ## 🔑 Civitai API_key（オプション）
# @markdown Civitaiからダウンロードする場合に必要
civitai_token = "" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 Custom Folder
# @markdown カスタムフォルダにダウンロードする場合のパスとURL
custom_folder_path = "" # @param {type:"string"}
custom_folder_urls = "" # @param {type:"string"}


# ========================================
# ダウンロード実行
# ========================================

print("=" * 70)
print("🚀 Starting model downloads...")
print("=" * 70)

# 各フォルダへのダウンロード
download_models_to_folder(diffusion_models_urls, "/content/ComfyUI/models/diffusion_models", civitai_token)
download_models_to_folder(text_encoders_urls, "/content/ComfyUI/models/text_encoders", civitai_token)
download_models_to_folder(vae_urls, "/content/ComfyUI/models/vae", civitai_token)
download_models_to_folder(clip_vision_urls, "/content/ComfyUI/models/clip_vision", civitai_token)
download_models_to_folder(loras_urls, "/content/ComfyUI/models/loras", civitai_token)
# download_models_to_folder(sam2_urls, "/content/ComfyUI/models/sam2", civitai_token) # Removed as requested
# download_models_to_folder(unet_urls, "/content/ComfyUI/models/unet", civitai_token) # Removed as requested
# download_models_to_folder(checkpoints_urls, "/content/ComfyUI/models/checkpoints", civitai_token) # Removed as requested
# download_models_to_folder(controlnet_urls, "/content/ComfyUI/models/controlnet", civitai_token) # Removed as requested
# download_models_to_folder(clip_urls, "/content/ComfyUI/models/clip", civitai_token) # Removed as requested
# download_models_to_folder(upscale_models_urls, "/content/ComfyUI/models/upscale_models", civitai_token) # Removed as requested
# download_models_to_folder(embeddings_urls, "/content/ComfyUI/models/embeddings", civitai_token) # Removed as requested

# カスタムフォルダ
if custom_folder_path and custom_folder_urls.strip():
    download_models_to_folder(custom_folder_urls, custom_folder_path, civitai_token)

print("=" * 70)
print("✓ All downloads completed!")
print("=" * 70)

# ========================================
# ComfyUI起動設定
# ========================================

use_cloudflare = False # @param {type:"boolean"}
use_interface_in_cell = False # @param {type:"boolean"}

if use_cloudflare:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb

    import subprocess
    import threading
    import time
    import socket
    import urllib.request

    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

      p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
          print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    clear_output()

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_L4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server

elif use_interface_in_cell:
    import threading
    import time
    import socket
    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      from google.colab import output
      output.serve_kernel_port_as_iframe(port, height=1024)
      clear_output()
      print("to open it in a window you can open this link here:")
      output.serve_kernel_port_as_window(port)

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_L4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server

else:
    import socket, time, threading
    from google.colab import output

    def link_thread(port):
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        clear_output()
        print("Click the link below to launch the comfyui interface")
        output.serve_kernel_port_as_window(port)

    # Start thread for port 8188
    threading.Thread(target=link_thread, daemon=True, args=(8188,)).start()

    if using_L4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server

# 🌱📋各種モデル_DL元リポジトリのメモ🌱
###　下記リンクのリポジトリから必要なモデルファイルのダウンロードURLを取得してきてください。
---
#🎬Wan2.2用 Diffusion Model
##＜t2v‗14B‗fp8＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- Wan2.2-T2V-A14B(High‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors)
- Wan2.2-T2V-A14B(Low‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors)
##＜i2v‗14B‗fp8＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- Wan2.2-I2V-A14B(High‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors)
- Wan2.2-I2V-A14B(Low‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors)

##＜t2v‗GGUF＞ ※ModelLoaderの変更が必要
####※下記リンクをそのまま使用可能（GGUF_Q8）
- Wan2.2-T2V-A14B-HighNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-T2V-A14B-HighNoise-Q8_0.gguf)
- Wan2.2-T2V-A14B-LowNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-T2V-A14B-LowNoise-Q8_0.gguf)

####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-T2V-A14B-GGUF：https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/tree/main
##＜i2v‗GGUF＞ ※ModelLoaderの変更が必要
####※下記リンクをそのまま使用可能（GGUF_Q8）
- Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf)
- Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf)

####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-I2V-A14B-GGUF：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/tree/main)

##＜Fun&Inp＞ ※動画の人物の動きを再現
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-Fun＆Inpl：(https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/tree/main/Fun)

##＜Animate‗GGUF＞ ※動画中の被写体をすげ替えor参照画像に反映
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-Animate-A14B-GGUF(https://huggingface.co/QuantStack/Wan2.2-Animate-14B-GGUF/tree/main)

#🎬Wan2.2用LoRA
##＜lightx2v_4steps（高速化）＞※High＆Low 別々に指定
####※下記リンクをそのまま使用可能
- LightX2v_t2v-4step_High：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_t2v_lightx2v_4steps_lora_v1.1_high_noise.safetensors)
- LightX2v_t2v-4step_Low：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_t2v_lightx2v_4steps_lora_v1.1_low_noise.safetensors)
- LightX2v_i2v-4step_High：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors)
- LightX2v_i2v-4step_Low：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors)
##＜lightx2v_Rank（高速化）＞※High＆Low共通
####※下記リンクをそのまま使用可能
- lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16：(https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors)
- lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16：(https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16.safetensors)

- LightX2v_t2v&i2v-Rank：(https://huggingface.co/Kijai/WanVideo_comfy/tree/main/Lightx2v)

##＜relight_LoRA（Animate用）＞ ※Wan2.2 Animateに必須のLoRA
####※下記リンクをそのまま使用可能
- Wan2.2_relight：(https://huggingface.co/Kijai/WanVideo_comfy/tree/main/LoRAs/Wan22_relight)

#🎬Wan2.2用VAE
##＜VAE＞
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.1_VAE.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/tree/main/split_files/vae)

#🎬Wan2.2用TextEncoder
##＜TextEncoder＞
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- umt5_xxl_fp16.safetensors：
(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/tree/main/split_files/text_encoders)

#🎬Wan2.2用ClipVision
##＜ClipVision＞
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- clip_vision_h.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/tree/main/split_files/clip_vision)

---
